In [67]:
import pandas as pd
import numpy as np
import os

# **Dataset : Fashion Product Images**

In [68]:
fashion_data = pd.read_csv('csv/fashion_product_images.csv', delimiter=',')

#Elegimos solo las columnas textuales
fashion_data = fashion_data[['gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName']]

fashion_data.head()

,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


**Ahora concatenamos y creamos los txt's**

In [69]:
"""# Ruta de la carpeta donde se guardarán los archivos de texto
output_folder = 'unprocessed_txt'

# Crea la carpeta si no existe
os.makedirs(output_folder, exist_ok=True)

# Itera a través de cada fila y guarda en un archivo de texto
for index, row in fashion_data.iterrows():
    # Convierte la fila en una cadena, separando los elementos por un espacio
    row_str = ' '.join(str(item) for item in row)
    
    # Crea un archivo de texto para cada fila en la carpeta específica
    file_path = os.path.join(output_folder, f'doc_{index + 1}.txt')
    with open(file_path, 'w') as file:
        file.write(row_str)"""

"# Ruta de la carpeta donde se guardarán los archivos de texto\noutput_folder = 'unprocessed_txt'\n\n# Crea la carpeta si no existe\nos.makedirs(output_folder, exist_ok=True)\n\n# Itera a través de cada fila y guarda en un archivo de texto\nfor index, row in fashion_data.iterrows():\n    # Convierte la fila en una cadena, separando los elementos por un espacio\n    row_str = ' '.join(str(item) for item in row)\n    \n    # Crea un archivo de texto para cada fila en la carpeta específica\n    file_path = os.path.join(output_folder, f'doc_{index + 1}.txt')\n    with open(file_path, 'w') as file:\n        file.write(row_str)"

## **Pre-procesamiento**

In [70]:
import nltk 
from nltk.corpus import stopwords #Stopwords
from nltk.stem import SnowballStemmer #Stemming

#Descargamos las stopwords
nltk.download('stopwords')

#Como todo el contenido esta en ingles, las stopwords tambien lo van a estar
stopwords = stopwords.words('english')

#De la misma forma como todo esta en ingles, el stemming tmb lo va a ser
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
"""directorio_entrada = 'unprocessed_txt'
directorio_salida = 'processed_txt'

# Crea la carpeta de salida si no existe
os.makedirs(directorio_salida, exist_ok=True)

contador_docs = 0

for filename in os.listdir(directorio_entrada):
    if filename.endswith(".txt"):

        contador_docs += 1

        filepath = os.path.join(directorio_entrada, filename)

        #Tokenizamos nuestro txt
        file_content = open(filepath, encoding="utf-8").read().lower()
        tokens = nltk.word_tokenize(file_content)

        #Filtramos para que no pertenezca a los stopwords o no sea un valor no alfanumerico (Stopwords / Valores raros)
        texto_filtrado = [word for word in tokens if not word in stopwords and word.isalnum()]

        #Hacemos Stemming en el idioma respectivo
        texto_filtrado = [stemmer.stem(w) for w in texto_filtrado]

        texto_procesado = " ".join(texto_filtrado)

        #Guardamos la lista unida en un archivo txt en el directorio de salida
        salida_filepath = os.path.join(directorio_salida, f"{filename}")
        
        #Guardamos el txt en el directorio de salida
        with open(salida_filepath, 'w', encoding="utf-8") as output_file:
            output_file.write(texto_procesado)
"""

'directorio_entrada = \'unprocessed_txt\'\ndirectorio_salida = \'processed_txt\'\n\n# Crea la carpeta de salida si no existe\nos.makedirs(directorio_salida, exist_ok=True)\n\ncontador_docs = 0\n\nfor filename in os.listdir(directorio_entrada):\n    if filename.endswith(".txt"):\n\n        contador_docs += 1\n\n        filepath = os.path.join(directorio_entrada, filename)\n\n        #Tokenizamos nuestro txt\n        file_content = open(filepath, encoding="utf-8").read().lower()\n        tokens = nltk.word_tokenize(file_content)\n\n        #Filtramos para que no pertenezca a los stopwords o no sea un valor no alfanumerico (Stopwords / Valores raros)\n        texto_filtrado = [word for word in tokens if not word in stopwords and word.isalnum()]\n\n        #Hacemos Stemming en el idioma respectivo\n        texto_filtrado = [stemmer.stem(w) for w in texto_filtrado]\n\n        texto_procesado = " ".join(texto_filtrado)\n\n        #Guardamos la lista unida en un archivo txt en el directorio d

# **Indice Invertido en Memoria Secundaria**

**Asocio cada termino a un ID de tamaño fijo**

In [ ]:
import sys
import ast
import re
import collections

from collections import OrderedDict


def sort_terms(term_postings_list):
    """ Sorts dictionary terms in alphabetical order """
    print(" -- Sorting terms...")
    sorted_dictionary = OrderedDict() # keep track of insertion order
    sorted_terms = sorted(term_postings_list)
    for term in sorted_terms:
        result = [int(docIds) for docIds in term_postings_list[term]]
        result_tftd = calculate_tftd(result)
        sorted_dictionary[term] = result_tftd
    return sorted_dictionary

def calculate_tftd(pl_with_duplicates):
    """ Add term frequency of term in each document """
    # print(pl_with_duplicates)
    counter = collections.Counter(pl_with_duplicates)
    pl_tftd = [[int(docId), counter[docId]] for docId in counter.keys()]
    return pl_tftd

def write_block_to_disk(term_postings_list, block_number):
    """ Writes index of the block (dictionary + postings list) to disk """
    # Define block
    base_path = 'index_blocks/'
    block_name = 'block-' + str(block_number) + '.txt'
    block = open(base_path + block_name, 'a+')
    print(" -- Writing term-positing list block: " + block_name + "...")
    # Write term : posting lists to block
    for index, term in enumerate(term_postings_list):
        # Term - Posting List Format
        # term:[docID1, docID2, docID3]
        # e.g. cat:[4,9,21,42]
        block.write(str(term) + ":" + str((term_postings_list[term])) + "\n")
    block.close()

In [72]:
def spimi_invert(documents, block_size_limit):
    """ Applies the Single-pass in-memory indexing algorithm """
    block_number = 0
    documents_count = len(documents)
    dictionary = {} # (term - postings list)
    for index, docID in enumerate(documents):
        for term in documents[docID]:
            # If term occurs for the first time
            if term not in dictionary:
                # Add term to dictionary, create new postings list, and add docID
                dictionary[term] = [docID]
            # else:
            #     # If term has a subsequent occurence
            #     if docID not in dictionary[term]:
            #         # Add a posting (docID) to the existing posting list of the term
            #         dictionary[term].append(docID)
            else:
                dictionary[term].append(docID)
        if sys.getsizeof(dictionary) > block_size_limit or (index == documents_count-1):
            temp_dict = sort_terms(dictionary)
            write_block_to_disk(temp_dict, block_number)
            temp_dict = {}
            block_number += 1
            dictionary = {}
    print("SPIMI invert complete!")

08e3c4c91ca5c58e479ea699df7b1a7e3743831ac17b27e1e967c7d74f41df23
646b1045ad1e525978a10d5a7b55f93440b821e514cd98fc43f4d2d1ccc11b20
511a79071666e78fa9283cc066ebafd2fd5e45e7f49705a34b4cdf023d818e67
33a0d09c376fe8b083b04d0a974d2f8577242ed703f66d5d7edac3541bf0826c
df108922e732f36d1805f839880815580356669b8d1ff8cd94bcb1ca34e177e1
16477688c0e00699c6cfa4497a3612d7e83c532062b64b250fed8908128ed548
2dc46af1b78c32dfe99dedfbdff6ca7f33e0f652716482f55b03622b67c06dea
c633eb52f63cec1d6b02d42448254cd801f5cbcaa254313c2795ce38bcd3a7fa
74dd730b5c36c1cb4fdbb4bce1764c57f260190f94ba52d877308332f1dfe363
20f65c28671b40937c5bf23acc7c6f37e5a5ec0622e347b57685725df5ba9e50
08e3c4c91ca5c58e479ea699df7b1a7e3743831ac17b27e1e967c7d74f41df23
df108922e732f36d1805f839880815580356669b8d1ff8cd94bcb1ca34e177e1
16477688c0e00699c6cfa4497a3612d7e83c532062b64b250fed8908128ed548
a52514252adf86066966790e947f72ef6ea6690a6852d5bb5480ee2eeb663395
